#### Zadanie 8

Przeprowadź analizę skupień za pomocą algorytmu dbscan na zbiorze danych z moduł sklearn.datasets https://scikit-learn.org/stable/datasets.html (np. iris lub breast_cancer)

Jeszcze raz popatrzmy na zbiór iris.

In [7]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris

iris_dataset = load_iris()
data = iris_dataset.data
target = iris_dataset.target
iris_df = pd.DataFrame(
    data=np.c_[data, target],  # np.c_ - numpy concatenate function used here to concatenate iris['data'] and iris['target'] arrays
    columns=iris_dataset.feature_names + ['target']
)
iris_df

# 0.0 - malignant
# 1.0 - benign

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2.0
146,6.3,2.5,5.0,1.9,2.0
147,6.5,3.0,5.2,2.0,2.0
148,6.2,3.4,5.4,2.3,2.0


In [8]:
y = iris_df.pop('target')
X = iris_df

In [9]:
y

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
145    2.0
146    2.0
147    2.0
148    2.0
149    2.0
Name: target, Length: 150, dtype: float64

In [10]:
X

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [11]:
X.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333
std,0.828066,0.435866,1.765298,0.762238
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


Poszczególne cechy nie różnią się bardzo od siebie wzajemnie. Wobec tego spróbujmy bez standaryzacji. Standaryzacja w tym przypadku raczej nam nie pomoże, a może pogorszyć wyniki.

**Uczenie modelu**

W jaki sposób dobierzemy średnicę sąsiedztwa oraz minimalną liczbę próbek w sąsiedztwie?

Spróbujmy użyć brute force.

In [12]:
from sklearn.cluster import DBSCAN

result_nr = 1
for i in range(1, 10):
    for j in range(4, 10):
        cluster = DBSCAN(eps=i/10, min_samples=j)
        y_dbscan = cluster.fit_predict(X)
        if len(set(y_dbscan)) >= 4:  # wyniki wyświetlamy tylko dla tych wartości parametrów, dla których algorytm wykrył 4 lub więcej skupis 
            # (z uwzględnieniem jednego skupiska jako outlierów). Dlaczego dopuszczamy większą liczbę klastrów? Ponieważ możliwa jest sytuacja, że
            # nadmiarowe klastry wykryte przez algorytm będą zaniedbywalne małe. Wtedy wyrzucimy te nadmiarowe traktując je jako szum i zostaniemy
            # z ładnym pogrupowaniem danych.
            print(f"Wynik {result_nr}")
            result_nr += 1
            print("-----------")
            print(f"eps: {i/10}")
            print(f"min_samples: {j}")
            df = pd.DataFrame({'labels': y, 'clusters': y_dbscan})
            print(pd.crosstab(df['labels'], df['clusters']))
            print("=====================================")

Wynik 1
-----------
eps: 0.2
min_samples: 4
clusters  -1   0   1   2
labels                  
0.0       32  10   8   0
1.0       46   0   0   4
2.0       50   0   0   0
Wynik 2
-----------
eps: 0.3
min_samples: 4
clusters  -1   0   1   2   3   4   5
labels                              
0.0       12  38   0   0   0   0   0
1.0       28   0   4  13   4   0   1
2.0       41   0   0   0   0   4   5
Wynik 3
-----------
eps: 0.3
min_samples: 5
clusters  -1   0   1   2
labels                  
0.0       13  37   0   0
1.0       37   0  12   1
2.0       46   0   0   4
Wynik 4
-----------
eps: 0.4
min_samples: 4
clusters  -1   0   1   2   3
labels                      
0.0        3  47   0   0   0
1.0        5   0  38   3   4
2.0       17   0   0  33   0
Wynik 5
-----------
eps: 0.4
min_samples: 5
clusters  -1   0   1   2   3
labels                      
0.0        4  46   0   0   0
1.0       11   0  36   3   0
2.0       17   0   0  11  22
Wynik 6
-----------
eps: 0.4
min_samples: 6
clusters  -

Widzimy, że wynik 4 daje najlepsze rezultaty i są one całkiem zadowalające. 

Zatem przyjmujemy: 
$$eps = {0.4}$$ $$min\_samples=4$$.

Jeszcze raz to samo dla przyjętych wartości parametrów.

In [13]:
from sklearn.cluster import DBSCAN

cluster = DBSCAN(eps=0.4, min_samples=4)
y_dbscan = cluster.fit_predict(X)  # trenujemy

In [14]:
y_dbscan[:10]  # obliczone numery klastrów dla poszczególnych punktów

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [15]:
df = pd.DataFrame({'labels': y, 'clusters': y_dbscan})
df

,labels,clusters
0,0.0,0
1,0.0,0
2,0.0,0
3,0.0,0
4,0.0,0
...,...,...
145,2.0,2
146,2.0,2
147,2.0,2
148,2.0,2


In [16]:
# Porównajmy predykcje (y_dbscan) z etykietami (y)

ct = pd.crosstab(df['labels'], df['clusters'])
ct

clusters,-1,0,1,2,3
labels,,,,,
0.0,3,47,0,0,0
1.0,5,0,38,3,4
2.0,17,0,0,33,0
